# Análise e Previsão de Churn de Clientes - TelecomX

Este notebook consolida todas as etapas do projeto de previsão de churn para a TelecomX, desde a análise exploratória de dados (EDA) até a otimização e avaliação de modelos de Machine Learning. O objetivo é identificar clientes com alta probabilidade de cancelar seus serviços, permitindo à empresa implementar estratégias de retenção proativas.

## 1. Configuração e Carregamento de Dados

Nesta seção, importamos as bibliotecas necessárias e carregamos o dataset de clientes da TelecomX. O dataset está em formato JSON e contém informações aninhadas que serão desaninhadas para facilitar a análise.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import json
from pathlib import Path
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix, precision_recall_curve
from imblearn.over_sampling import SMOTE
import joblib
from scipy.stats import uniform, randint

# Configurações de visualização
sns.set(style="whitegrid")

# Caminhos para dados e modelos
DATA_RAW_PATH = Path('../data/raw/Telco-Customer-Churn.json')
DATA_PROCESSED_PATH = Path('../data/processed')
MODELS_PATH = Path('../models')
REPORTS_PATH = Path('../reports')
FIG_DIR = REPORTS_PATH / "figures"

# Criação de diretórios se não existirem
DATA_PROCESSED_PATH.mkdir(parents=True, exist_ok=True)
MODELS_PATH.mkdir(parents=True, exist_ok=True)
REPORTS_PATH.mkdir(parents=True, exist_ok=True)
FIG_DIR.mkdir(parents=True, exist_ok=True)

# Função para carregar e desaninhar dados JSON
def load_and_flatten_json(path: Path) -> pd.DataFrame:
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    df = pd.json_normalize(data)
    return df

df = load_and_flatten_json(DATA_RAW_PATH)
df.head()

## 2. Análise Exploratória de Dados (EDA)

Esta seção foca na compreensão inicial dos dados, suas distribuições, valores ausentes e a relação entre as variáveis e a variável alvo (churn).

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# Preparar a variável alvo para EDA
df_eda = df.copy()
df_eda["Churn"] = df_eda["Churn"].replace({"": np.nan})
df_eda["churn_flag"] = df_eda["Churn"].map({"Yes": 1, "No": 0})

# Distribuição de Churn
plt.figure(figsize=(6,4))
sns.countplot(x="Churn", data=df_eda, order=["No", "Yes"])
plt.title("Distribuição de Churn (alvo)")
plt.xlabel("Churn")
plt.ylabel("Contagem")
plt.tight_layout()
plt.savefig(FIG_DIR / "target_churn_distribution.png", dpi=150)
plt.show()

# Conversões numéricas de cobranças (podem vir como string)
for c in ["account.Charges.Monthly", "account.Charges.Total"]:
    if c in df_eda.columns:
        df_eda[c] = pd.to_numeric(df_eda[c], errors="coerce")

# Tenure
if "customer.tenure" in df_eda.columns:
    df_eda["customer.tenure"] = pd.to_numeric(df_eda["customer.tenure"], errors="coerce")

# Plotar distribuição de cobrança mensal por churn
plt.figure(figsize=(7,4))
sns.kdeplot(data=df_eda.dropna(subset=["account.Charges.Monthly", "Churn"]), x="account.Charges.Monthly", hue="Churn", common_norm=False, fill=True)
plt.title("Distribuição de cobrança mensal por churn")
plt.tight_layout()
plt.savefig(FIG_DIR / "kde_monthly_by_churn.png", dpi=150)
plt.show()

# Plotar distribuição de tenure por churn
plt.figure(figsize=(7,4))
sns.kdeplot(data=df_eda.dropna(subset=["customer.tenure", "Churn"]), x="customer.tenure", hue="Churn", common_norm=False, fill=True)
plt.title("Distribuição de tenure por churn")
plt.tight_layout()
plt.savefig(FIG_DIR / "kde_tenure_by_churn.png", dpi=150)
plt.show()

# Função auxiliar para plotar taxa de churn por categoria
def plot_categorical_churn_rate(df_plot: pd.DataFrame, col: str, title: str, fname: str):
    if col not in df_plot.columns:
        return
    tmp = (
        df_plot.dropna(subset=[col, "churn_flag"])
          .groupby(col)["churn_flag"].mean()
          .sort_values(ascending=False)
    )
    plt.figure(figsize=(8,4))
sns.barplot(x=tmp.index, y=tmp.values, color="#3366cc")
    plt.title(title)
    plt.ylabel("Taxa de churn (média)")
    plt.xlabel(col)
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.savefig(FIG_DIR / fname, dpi=150)
    plt.show()

# Plotar churn médio por tipo de contrato
plot_categorical_churn_rate(df_eda, "account.Contract", "Churn médio por tipo de contrato", "rate_by_contract.png")

# Plotar churn médio por tipo de serviço de internet
plot_categorical_churn_rate(df_eda, "internet.InternetService", "Churn médio por tipo de internet", "rate_by_internet_service.png")

# Sumário de valores ausentes
missing_by_col = df_eda.isna().mean().sort_values(ascending=False).head(15)
print("
Top 15 colunas com mais valores ausentes:")
print(missing_by_col)

## 3. Pré-processamento e Engenharia de Features

Nesta etapa, os dados são limpos, transformados e novas features são criadas para melhorar o desempenho dos modelos de Machine Learning. Isso inclui o tratamento de valores ausentes, conversão de tipos de dados e a criação de variáveis que capturam informações mais relevantes.

In [ ]:
def clean_and_engineer_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # Convert 'TotalCharges' to numeric, coercing errors to NaN
    df["account.Charges.Total"] = pd.to_numeric(df["account.Charges.Total"], errors="coerce")

    # Fill missing 'TotalCharges' with 0 (assuming new customers or very low usage)
    df["account.Charges.Total"] = df["account.Charges.Total"].fillna(0)

    # Convert 'Churn' to binary (1 for Yes, 0 for No), handle empty strings as NaN
    df["Churn"] = df["Churn"].replace({"": np.nan})
    df = df.dropna(subset=["Churn"]) # Drop rows where Churn is NaN
    df["churn_flag"] = df["Churn"].map({"Yes": 1, "No": 0})

    # Drop original 'Churn' column if 'churn_flag' is created
    df = df.drop(columns=["Churn"], errors="ignore")

    # Feature Engineering
    # AvgMonthlyCost = TotalCharges / Tenure (handle division by zero for tenure=0)
    df["AvgMonthlyCost"] = df.apply(lambda row: row["account.Charges.Total"] / row["customer.tenure"] if row["customer.tenure"] != 0 else 0, axis=1)

    # Total_Servicos: count of additional services
    service_cols = [
        "phone.MultipleLines", "internet.OnlineSecurity", "internet.OnlineBackup",
        "internet.DeviceProtection", "internet.TechSupport", "internet.StreamingTV",
        "internet.StreamingMovies"
    ]
    # Replace 'No phone service' and 'No internet service' with 'No' for consistency in counting
    for col in service_cols:
        if col in df.columns:
            df[col] = df[col].replace({"No phone service": "No", "No internet service": "No"})

    df["Total_Servicos"] = df[service_cols].apply(lambda row: sum(row == "Yes"), axis=1)

    # Idoso_Com_Dependentes: 1 if SeniorCitizen is 1 and Dependents is 'Yes'
    df["Idoso_Com_Dependentes"] = ((df["customer.SeniorCitizen"] == 1) & (df["customer.Dependents"] == "Yes")).astype(int)

    # Drop irrelevant columns identified from EDA or common sense
    cols_to_drop = [
        "customerID", # Identifier
    ]
    df = df.drop(columns=cols_to_drop, errors="ignore")

    return df

df_processed = clean_and_engineer_features(df)
df_processed.head()

## 4. Divisão de Dados e Pré-processamento com Pipeline

Dividimos os dados em conjuntos de treino e teste e aplicamos transformações (escalonamento para numéricos e One-Hot Encoding para categóricos) usando um `ColumnTransformer` dentro de um `Pipeline` para garantir que o pré-processamento seja aplicado de forma consistente.

In [ ]:
def split_data(df: pd.DataFrame, target_col: str = "churn_flag", test_size: float = 0.3, random_state: int = 42):
    X = df.drop(columns=[target_col])
    y = df[target_col]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = split_data(df_processed)

# Identificar colunas categóricas e numéricas
categorical_cols = X_train.select_dtypes(include=["object", "bool"]).columns
numerical_cols = X_train.select_dtypes(include=np.number).columns

# Criar pipelines de pré-processamento para features numéricas e categóricas
numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown="ignore", drop="first")

# Criar um ColumnTransformer para aplicar diferentes transformações a diferentes colunas
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numerical_transformer, numerical_cols),
        ("cat", categorical_transformer, categorical_cols),
    ],
    remainder="passthrough" # Manter outras colunas (se houver) que não são transformadas
)

# Fit o preprocessor nos dados de treino
preprocessor.fit(X_train)

# Transformar dados de treino e teste
X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# Salvar o preprocessor ajustado
joblib.dump(preprocessor, MODELS_PATH / "preprocessor.pkl")

print("Pré-processamento concluído. Dados transformados e pré-processador salvo.")

## 5. Modelagem e Avaliação

Nesta seção, treinamos e avaliamos diferentes modelos de Machine Learning (Regressão Logística, Random Forest, XGBoost) para a previsão de churn. As métricas de avaliação incluem Acurácia, Precisão, Recall, F1-Score e AUC, com foco especial em Recall e AUC para este problema de negócio.

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

def train_and_evaluate_model(model, X_train, y_train, X_test, y_test, model_name):
    print(f"
--- Treinando e Avaliando {model_name} ---")

    # Aplicar SMOTE apenas nos dados de treino
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

    model.fit(X_train_resampled, y_train_resampled)
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_proba)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print(f"ROC AUC: {roc_auc:.4f}")
    print("
Classification Report:")
    print(classification_report(y_test, y_pred))
    print("
Confusion Matrix:")
    print(confusion_matrix(y_test, y_pred))

    # Cross-validation
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = cross_val_score(model, X_train_resampled, y_train_resampled, cv=cv, scoring="roc_auc")
    print(f"
Cross-validation ROC AUC scores: {cv_scores}")
    print(f"Mean CV ROC AUC: {np.mean(cv_scores):.4f} (+/- {np.std(cv_scores):.4f})")

    # Salvar modelo
    joblib.dump(model, MODELS_PATH / f"model_{model_name.lower().replace(' ', '_')}.pkl")

    return {
        "model_name": model_name,
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "roc_auc": roc_auc,
        "mean_cv_roc_auc": np.mean(cv_scores)
    }

models = {
    "Logistic Regression": LogisticRegression(random_state=42, solver="liblinear"),
    "Random Forest": RandomForestClassifier(random_state=42),
    "XGBoost": XGBClassifier(random_state=42, use_label_encoder=False, eval_metric="logloss")
}

results = []
for name, model in models.items():
    result = train_and_evaluate_model(model, X_train_processed, y_train, X_test_processed, y_test, name)
    results.append(result)

results_df = pd.DataFrame(results)
results_df.to_markdown(REPORTS_PATH / "model_evaluation_summary.md", index=False)
print(f"
Sumário da avaliação dos modelos salvo em {REPORTS_PATH / 'model_evaluation_summary.md'}")
print(results_df)

## 6. Otimização do Modelo e Análise de Resultados

Esta seção foca na otimização do modelo selecionado (Regressão Logística, devido ao seu bom recall e interpretabilidade) através do ajuste do threshold de classificação e na otimização de hiperparâmetros para o Random Forest como exemplo de melhoria de desempenho.

In [ ]:
# Carregar o modelo de Regressão Logística treinado
model_lr = joblib.load(MODELS_PATH / "model_logistic_regression.pkl")

def find_optimal_threshold(model, X_test, y_test, target_recall=0.8):
    y_proba = model.predict_proba(X_test)[:, 1]
    precisions, recalls, thresholds = precision_recall_curve(y_test, y_proba)

    optimal_threshold = 0.5 # Default
    for i in range(len(thresholds)):
        if recalls[i] >= target_recall:
            optimal_threshold = thresholds[i]
            break

    print(f"Threshold para recall >= {target_recall}: {optimal_threshold:.4f}")

    # Avaliar com o threshold otimizado
    y_pred_optimal = (y_proba >= optimal_threshold).astype(int)
    print("
Classification Report com Threshold Otimizado:")
    print(classification_report(y_test, y_pred_optimal))
    print("
Confusion Matrix com Threshold Otimizado:")
    print(confusion_matrix(y_test, y_pred_optimal))

    return optimal_threshold

print("
### Ajuste de Threshold para Regressão Logística ###")
optimal_threshold_lr = find_optimal_threshold(model_lr, X_test_processed, y_test, target_recall=0.8)
joblib.dump(optimal_threshold_lr, MODELS_PATH / "optimal_threshold_lr.pkl")

def hyperparameter_tuning(model, X_train, y_train, param_distributions, n_iter=10, cv=5):
    print("
--- Otimização de Hiperparâmetros com RandomizedSearchCV ---")
    random_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_distributions,
        n_iter=n_iter,
        cv=cv,
        scoring="roc_auc",
        random_state=42,
        n_jobs=-1, # Usar todos os cores disponíveis
        verbose=1
    )
    random_search.fit(X_train, y_train)
    print(f"Melhores hiperparâmetros: {random_search.best_params_}")
    print(f"Melhor ROC AUC (CV): {random_search.best_score_:.4f}")
    return random_search.best_estimator_

# Aplicar SMOTE para o conjunto de treino antes do tuning
smote = SMOTE(random_state=42)
X_train_resampled_tuning, y_train_resampled_tuning = smote.fit_resample(X_train_processed, y_train)

print("
### Otimização de Hiperparâmetros para Random Forest ###")
rf_model = RandomForestClassifier(random_state=42)
param_dist_rf = {
    "n_estimators": randint(100, 500),
    "max_features": ["sqrt", "log2"],
    "max_depth": randint(5, 30),
    "min_samples_split": randint(2, 20),
    "min_samples_leaf": randint(1, 20),
    "bootstrap": [True, False]
}
best_rf_model = hyperparameter_tuning(rf_model, X_train_resampled_tuning, y_train_resampled_tuning, param_dist_rf, n_iter=20)
joblib.dump(best_rf_model, MODELS_PATH / "best_random_forest_model.pkl")

print("Otimização do modelo concluída.")

## 7. Conclusão e Próximos Passos

Este projeto demonstrou a construção de um pipeline completo para previsão de churn, desde a ingestão de dados até a otimização de modelos. O modelo de Regressão Logística, com seu threshold ajustado para priorizar o recall, mostrou-se promissor para identificar clientes em risco.

**Próximos Passos:**
*   **Implementação em Produção:** Integrar o modelo e o pré-processador em um sistema de produção para inferência em tempo real ou em lotes.
*   **Monitoramento Contínuo:** Acompanhar o desempenho do modelo em produção e re-treiná-lo periodicamente com novos dados.
*   **Análise de Custo-Benefício:** Avaliar o impacto financeiro das ações de retenção baseadas nas previsões do modelo.
*   **Exploração de Outros Modelos:** Testar modelos mais avançados ou ensembles para potencial melhoria de desempenho.
*   **Engenharia de Features Avançada:** Investigar a criação de features mais complexas ou o uso de técnicas de seleção de features.
